## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models, Model

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data

X_Columns = ['DistanceFromHome','HourlyRate','JobLevel','JobSatisfaction','PerformanceRating','PercentSalaryHike','YearsInCurrentRole','YearsSinceLastPromotion','OverTime','WorkLifeBalance']

# Create X_df using your selected columns
X = attrition_df[X_Columns]

# Show the data types for X_df
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DistanceFromHome         1470 non-null   int64 
 1   HourlyRate               1470 non-null   int64 
 2   JobLevel                 1470 non-null   int64 
 3   JobSatisfaction          1470 non-null   int64 
 4   PerformanceRating        1470 non-null   int64 
 5   PercentSalaryHike        1470 non-null   int64 
 6   YearsInCurrentRole       1470 non-null   int64 
 7   YearsSinceLastPromotion  1470 non-null   int64 
 8   OverTime                 1470 non-null   object
 9   WorkLifeBalance          1470 non-null   int64 
dtypes: int64(9), object(1)
memory usage: 115.0+ KB


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_df_train, y_df_test = train_test_split(X,y_df)


In [6]:
#function to update the values of OverTime column to 1 and 0
def encode(OverTime):
    """
    This function encodes the method of purchases to 2 for "Retail"
    and 1 for Hotel/Restuarant/Cafe.
    """
    if OverTime == "Yes":
        return 1
    else:
        return 0

In [7]:
#Run function on Train and Test sets
X_train['OverTime'] = X_train['OverTime'].apply(encode)
X_test['OverTime'] = X_test['OverTime'].apply(encode)

In [8]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].astype('int')
X_test['OverTime'] = X_test['OverTime'].astype('int')

In [9]:
X_train['OverTime'].value_counts()

OverTime
0    790
1    312
Name: count, dtype: int64

In [10]:
# Create a StandardScaler and fit the train data
scaler = StandardScaler().fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [11]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import  OneHotEncoder

department_encoder = OneHotEncoder(sparse_output=False)
# Fit the encoder to the training data
department_encoded = department_encoder.fit(y_df_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
dept_encoded_train = department_encoded.transform(y_df_train[['Department']])
dept_encoded_test = department_encoded.transform(y_df_test[['Department']])




In [12]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoded = attrition_encoder.fit(y_df_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
att_encoded_train = attrition_encoded.transform(y_df_train[['Attrition']])
att_encoded_test = attrition_encoded.transform(y_df_test[['Attrition']])


## Create, Compile, and Train the Model

In [13]:
#Find out columns
input_nodes = len(X.columns)
# Create the input layer
input_layer = layers.Input(shape=(input_nodes), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu',name='shared2')(shared_layer1)

In [14]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch = layers.Dense(32,activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_branch)


In [15]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = layers.Dense(32,activation='relu', name="attrition_hidden")(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_branch)


In [16]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output':'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_features (InputLayer  [(None, 10)]                 0         []                            
 )                                                                                                
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input_features[0][0]']      
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']         

In [17]:
# Train the model
model.fit(
    X_train_scaled,
    {'department_output': dept_encoded_train,'attrition_output': att_encoded_train},
    epochs = 50,
    batch_size =32,
    validation_split = 0.2
)


Epoch 1/50
28/28 [==============================] - 7s 35ms/step - loss: 1.5490 - department_output_loss: 0.9334 - attrition_output_loss: 0.6156 - department_output_accuracy: 0.5505 - attrition_output_accuracy: 0.7037 - val_loss: 1.2913 - val_department_output_loss: 0.7811 - val_attrition_output_loss: 0.5101 - val_department_output_accuracy: 0.6244 - val_attrition_output_accuracy: 0.8145
Epoch 2/50
28/28 [==============================] - 0s 5ms/step - loss: 1.2388 - department_output_loss: 0.7847 - attrition_output_loss: 0.4541 - department_output_accuracy: 0.6493 - attrition_output_accuracy: 0.8434 - val_loss: 1.2469 - val_department_output_loss: 0.7760 - val_attrition_output_loss: 0.4709 - val_department_output_accuracy: 0.6244 - val_attrition_output_accuracy: 0.8145
Epoch 3/50
28/28 [==============================] - 0s 5ms/step - loss: 1.1689 - department_output_loss: 0.7589 - attrition_output_loss: 0.4101 - department_output_accuracy: 0.6493 - attrition_output_accuracy: 0.8434 - 

In [20]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': dept_encoded_test,'attrition_output': att_encoded_test})
test_results

12/12 [==============================] - 0s 5ms/step - loss: 2.1960 - department_output_loss: 1.5607 - attrition_output_loss: 0.6353 - department_output_accuracy: 0.6033 - attrition_output_accuracy: 0.8207


[2.196016311645508,
 1.5606777667999268,
 0.6353384256362915,
 0.60326087474823,
 0.820652186870575]

In [22]:
# Print the accuracy for both department and attrition
print(f"Deparment Accuracy: {test_results[3]}")
print(f"Attrition Accuracy: {test_results[4]}")

Deparment Accuracy: 0.60326087474823
Attrition Accuracy: 0.820652186870575


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. I think loss may be a better option to evaluate the model, specially for Department the loss is very high which and low accuracy which means that this target may not be useful with our set of data. For attrition the loss and accuracy are better which also assists in that department is not a good target predictor with our data. And that we maybe can use a different set of features.

2. The functions chosen for the output layers is sigmoid for Attrition, this was chosen because is a binary output of yes or no.
For department output softmax was chosen as the department is not binary as multiple departments can exist.

3. This model could be improved with better features, maybe narrow down to the few relevant ones, instead of using a broad set of data that may not be useful. Also department may not be a good target, as that is not likely relevant as whether a worker stays or not.